In the problem http://puzzlor.com/2008-08_MarkovsPrison.html, we are asked to find an escape route for a prisoner that minimizes their probability of being caught.  Since the guards move from room to room with given probabilities independent of their historical states (and because the word is in the title of the puzzle) we can use a Markov chain to determine the safest path for the prisoner to take.  The guards have been moving through the prison for years, which means that the Markov chain will be well-mixed, so we can make use of the long term transition probabilities that correspond to the left eigenvectors of the transition matrix.   Since there are 16 rooms, we have 16 possible states for each guard, and our transition matrices will be 16 x 16.  We can either find the left eigenvector for each matrix associated to the eigenvalue \lambda = 1, or simply take a large power of each matrix to approximate the limiting distribution.  Once we have our limiting distributions, we need to find the path from 1 to 16 that constitutes the smallest probability of getting caught.

In [1]:
import numpy as np
import scipy.sparse as sp
np.set_printoptions(precision = 2)

In [2]:
# create the transition matrix for the first guard
G1 = np.array([[.4, .2, 0, 0, .4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [.2, .2, .2, 0, 0,.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, .2, .2, .2, 0, 0,.4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, .2, .4, 0, 0, 0, .4, 0, 0, 0, 0, 0, 0, 0, 0],
               [.2, 0, 0, 0, .2, .2, 0, 0, .4, 0, 0, 0, 0, 0, 0, 0],
               [0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0, 0, 0, 0, 0, 0],
               [0, 0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0, 0, 0, 0, 0],
               [0, 0, 0, .2, 0, 0, .2, .2, 0, 0, 0, .4, 0, 0, 0, 0],
               [0, 0, 0, 0, .2, 0, 0, 0, .2, .2, 0, 0, .4, 0, 0, 0],
               [0, 0, 0, 0, 0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0, 0],
               [0, 0, 0, 0, 0, 0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0],
               [0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .2, 0, 0, 0, .4],
               [0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, 0, .6, .2, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .4, .2, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .4, .2],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .6]])

In [3]:
G1.shape

(16, 16)

In [4]:
# check that we are indeed a stochastic matrix
np.sum(G1,axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.])

In [5]:
G2 = np.array([[.6, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [.2, .4, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, .2, .4, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, .2, .7, 0, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0],
               [.4, 0, 0, 0, .2, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0],
               [0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0],
               [0, 0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0, 0, 0, 0, 0],
               [0, 0, 0, .4, 0, 0, .2, .3, 0, 0, 0, .1, 0, 0, 0, 0],
               [0, 0, 0, 0, .4, 0, 0, 0, .2, .3, 0, 0, .1, 0, 0, 0],
               [0, 0, 0, 0, 0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0, 0],
               [0, 0, 0, 0, 0, 0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0],
               [0, 0, 0, 0, 0, 0, 0, .4, 0, 0, .2, .3, 0, 0, 0, .1],
               [0, 0, 0, 0, 0, 0, 0, 0, .4, 0, 0, 0, .3, .3, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, .4, 0, 0, .2, .1, .3, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0,  0, .4, 0, 0, .2, .1, .3],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, .4, 0, 0, .2, .4]])

In [6]:
G2.shape

(16, 16)

In [7]:
np.sum(G2,axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.])

In [8]:
# we can approximate by taking a large power of the matrix
G1_steady_state = np.linalg.matrix_power(G1,1000)

In [9]:
G2_steady_state = np.linalg.matrix_power(G2,1000)

In [10]:
G2_steady_state.sum()

16.00000000000021

now that we have our limiting distributions, we can create a mask using the original costs (to prevent us from hopping between non adjacent rooms) and use Dijkstra's method to find the shortest path from 1 to 16, with the edge costs being the probability of a guard being in the rooms

In [11]:
soln = sp.csgraph.dijkstra(csgraph=G1, indices=np.array([0,15]),return_predecessors = True)

In [12]:
len(soln)

2

In [13]:
soln[0]

array([[ 0. ,  0.2,  0.4,  0.6,  0.4,  0.6,  0.8,  1. ,  0.8,  1. ,  1.2,
         1.4,  1.2,  1.4,  1.6,  1.8],
       [ 1.2,  1. ,  0.8,  0.6,  1. ,  0.8,  0.6,  0.4,  0.8,  0.6,  0.4,
         0.2,  0.6,  0.4,  0.2,  0. ]])

In [14]:
soln[1][0]

array([-9999,     0,     1,     2,     0,     1,     2,     3,     4,
           5,     9,     7,     8,     9,    13,    11], dtype=int32)

In [16]:
captureProbabilities = G1 + G2 - G1*G2
captureProbabilities   #I'm a bit suspicious of this

array([[ 0.76,  0.44,  0.  ,  0.  ,  0.46,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.36,  0.52,  0.44,  0.  ,  0.  ,  0.46,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.36,  0.52,  0.44,  0.  ,  0.  ,  0.46,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.36,  0.82,  0.  ,  0.  ,  0.  ,  0.46,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.52,  0.  ,  0.  ,  0.  ,  0.36,  0.44,  0.  ,  0.  ,  0.46,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.52,  0.  ,  0.  ,  0.36,  0.  ,  0.44,  0.  ,  0.  ,
         0.46,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.52,  0.  ,  0.  ,  0.36,  0.  ,  0.44,  0.  ,
         0.  ,  0.46,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.52,  0.  ,  0.  ,  0.36,  0.44,  0.  ,
         0.  ,  0.  ,  0.4

In [17]:
soln = sp.csgraph.dijkstra(csgraph=captureProbabilities, indices=np.array([0,15]), return_predecessors = True)

In [18]:
soln

(array([[ 0.  ,  0.44,  0.88,  1.32,  0.46,  0.9 ,  1.34,  1.78,  0.92,
          1.36,  1.8 ,  2.24,  1.38,  1.82,  2.26,  2.7 ],
        [ 2.64,  2.28,  1.92,  1.56,  2.12,  1.76,  1.4 ,  1.04,  1.6 ,
          1.24,  0.88,  0.52,  1.08,  0.72,  0.36,  0.  ]]),
 array([[-9999,     0,     1,     2,     0,     1,     2,     6,     4,
             5,     6,     7,     8,     9,    10,    11],
        [    4,     5,     6,     7,     5,     6,    10,    11,    12,
            13,    14,    15,    13,    14,    15, -9999]], dtype=int32))

In [27]:
next_node = 15
nodes = []
while next_node != 0:
    nodes.append(next_node)
    next_node = soln[1][0][next_node]

In [33]:
nodes.reverse() #hmm, no

In [36]:
for node in nodes:
    print node+1

2
3
7
8
12
16
